# 02 Predicting Armed Conflict Using Protest Data - Analysis

<b>Notebook description:</b> This notebook belongs to the replication files for the "Predicting Armed Conflict Using Protest Data" article. The notebook loads the data extracted from the VIEWS database (see "predicting_armed_conflict_using_protest_data_01_queries_data.ipynb").

<b>Note:</b> The notebook requires the user to check and install the required packages listed under [Loading modules](#modules). 

## Overview
* [Importing modules](#modules)
* [Defining folder structure](#define_folders)
* [Specifying global parameters](#specify_parameters)
* [Training models](#trainpredict)
* [Evaluation](#evaluation)

## Loading modules<a class="anchor" id="modules"></a>

In [ ]:
# Basics
import numpy as np
import pandas as pd
import geopandas as gpd

# Plot
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook

# Models
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Views-related packages 
import views_dataviz
from views_dataviz.map import mapper, utils
from views_stepshift import Period,Downsampling #(https://pypi.org/project/views-stepshift/)
from views_stepshift import Model,Ensemble
from views_stepshift.datautils import assign_into_df

# Mapper (https://pypi.org/project/views-mapper2/)
from views_mapper2.mapper2 import *
from views_mapper2.BBoxWriter import *
from views_mapper2.dictionary_writer import *
from views_mapper2.label_writer import *

# Evaluation
from sklearn.metrics import precision_recall_curve
from sklearn.inspection import plot_partial_dependence
from sklearn.inspection import partial_dependence
import sepplotlib as spl

# Additional transforms
import predicting_armed_conflict_using_protest_data_transforms as transforms
import predicting_armed_conflict_using_protest_data_models as organize 
import predicting_armed_conflict_using_protest_data_utils as utils 
import predicting_armed_conflict_using_protest_data_eval as evaltools

# Other packages
import importlib
import os
import yaml
import pickle 
from patsy import dmatrices
import random
from time import time
from datetime import datetime

## Defining folder structure<a class="anchor" id="define_folders"></a>

To follow the replication step by step, we recommend you to download and store the folder structure available here https://www.dropbox.com/sh/jiyjo6ic12mv6j6/AABogy4M1nNBtJTgvDysQcO7a?dl=0.   

In [ ]:
# Add your username 
username= 'maxle647' #'yourusername'

# Define path.
folder_path = f'/Users/{username}/Dropbox (ViEWS)/Protest article replication' #protest_views3'  # Change path
print('Folder path:', folder_path)


In [ ]:
# Create folders if they do not yet exist. 

if not os.path.isdir(folder_path):
    os.makedirs(folder_path)

# Set up directory for output
folder_path = os.path.join(folder_path, '{sub}')

# Define output paths
output_paths = {
    'descriptives': folder_path.format(sub=f'descriptives'),
    'evaluation': folder_path.format(sub=f'evaluation'),
    'summary_tables':folder_path.format(sub=f'summary_tables'),
    'predictions':folder_path.format(sub=f'predictions'),
    'maps':folder_path.format(sub=f'maps'),
    'models':folder_path.format(sub=f'models'),
    'data':folder_path.format(sub=f'data'),

    # Sub folders 
    'scores_tables': os.path.join(folder_path.format(sub=f"evaluation"), "scores_tables"),
    'coord_plots': os.path.join(folder_path.format(sub=f"evaluation"), "coord_plots"),
    'bootstrapped': os.path.join(folder_path.format(sub=f"evaluation"), "bootstrapped"),
    'pr_curves': os.path.join(folder_path.format(sub=f"evaluation"), "pr_curves"),
    'features': os.path.join(folder_path.format(sub=f"evaluation"), "features"),
    'bisep': os.path.join(folder_path.format(sub=f"evaluation"), "bisep"),

}

# Create new folders if they do not already exist.
for k, v in output_paths.items():
    if not os.path.isdir(v):
        os.makedirs(v)

## Specifying global parameters <a class="anchor" id="specify_parameters"></a>

In [ ]:
# Change here the outcome of interest to run the analysis for onset and the new periods.
# Possible ids to choose from: 'incidence' (main analysis), 'onset', 'incidence_np' (adjusted period), 'onset_np' (adjusted period)
run_outcome = 'incidence' 

In [ ]:
# Define the outcome variable
depvar = 'ged_sb_dummy_dep'

In [ ]:
# Define here whether to retrain the models from scratch or whether to use the pre-trained model objects used for the analysis. 
train = False
evaluate = True

## Load data

In [ ]:
# Load data.
with open(os.path.join(output_paths['data'], f"data_dict_incidence.p"), 'rb') as fp:
    datasets = pickle.load(fp)

## Define Regressors, Downsampling and Periods

In [ ]:
# Define random forest hyper-parameters.
nj=14
n_estimators=500

rf_classifier = RandomForestClassifier(n_jobs=nj, n_estimators=n_estimators, random_state=1308)

In [ ]:
# Define downsampling strategy.
downsampling = Downsampling(share_positive = 1.0, share_negative = 0.3)

In [ ]:
# Define steps. 
steps = [3,6,12,36]

In [ ]:
# Define periods. 
periods = [
    Period(name="A",train_start=205,train_end=408,predict_start=409,predict_end=444),
    Period(name="B",train_start=205,train_end=444,predict_start=445,predict_end=480),
]

adj_periods = [
    Period(name="A",train_start=205,train_end=408-36,predict_start=409-36,predict_end=444-36),
    Period(name="B",train_start=205,train_end=444-36,predict_start=445-36,predict_end=480-36),
]

In [ ]:
# Define which periods to train/test on. 
# Possible ids incldude 'periods', 'adj_periods'
periods_model = periods 

In [ ]:
# Overview.
print('Steps', steps)
print('Downsampling', downsampling)
print('Periods', periods_model)

In [ ]:
# Show model definitions. 
for df in datasets:
    print(df['Name'])

## Specify Models

In [ ]:
# Check parameters.
print('Dependent variable:', depvar, ', Outcome:', run_outcome)

In [ ]:
# Define models to be trained. 
models_to_train = [
    'baseline_simple',
    'econ_nat_bl',
    'econ_full_bl',
    'inst_elecdemo_bl',
    'inst_civlib_bl',
    'inst_elect_bl',
    'inst_devi_bl',
    'pr_naive_bl',
    'pr_dynamic_loc_bl',
    'pr_dynamic_nat_bl',
    'pr_elecdemo_bl',
    'pr_civlib_bl',
    'pr_elect_bl',
    'pr_devi_bl',
    'pr_econ_nat_bl',
    'pr_econ_full_bl',
]

In [ ]:
# Create list of models based on the previously defined parameters.
ModelList = []

with open('featlist_protest_paper.yaml', 'r') as file:
    full_featlist = yaml.safe_load(file)
    
for df in datasets:
    for mname in models_to_train:
        if df['Name'] == mname:
            print(mname)
            
            for feats,colname in zip(full_featlist.keys(),full_featlist.values()):
                if df['Name'] == feats:
                    featlist_dep = list(df['df'][colname].columns)
                    featlist_f = [x for x in featlist_dep if x != f'{depvar}']
            
            if 'incidence' in run_outcome:
                print('incidence')
                ModelList.append(
                    Model(
                        name = f'protest_{mname}_{run_outcome}',
                        col_outcome = depvar,
                        cols_features = featlist_f,
                        periods = periods_model,
                        steps = steps,
                        outcome_type = "prob",
                        downsampling =  downsampling,
                        estimator = rf_classifier,
                        dir_storage = output_paths['models'],
                    )
                )
            
            if 'onset' in run_outcome:
                print('onset')
                ModelList.append(
                    Model(
                        name = f'protest_{mname}_{run_outcome}',
                        col_outcome = depvar,
                        cols_features = featlist_f,
                        periods = periods_model,
                        steps = steps,
                        outcome_type = "prob",
                        downsampling =  downsampling,
                        onset_outcome= True,
                        onset_window= 6, 
                        estimator = rf_classifier,
                        dir_storage = output_paths['models'],
                    )
                )
                

In [ ]:
for model in ModelList:
    print(model.name)
    print(model.col_outcome)

## Train, Calibrate, Predict & Evaluate <a class="anchor" id="trainpredict"></a>

In [ ]:
# Saving or Loading - dependent on parameter at the top of the notebook.
if train:
    save_preds = True
else:
    save_preds = False
    
print(save_preds)

In [ ]:
# Train.
if train:
    random.seed(1308)

    start_time = datetime.now()
    for df in datasets:
        for model in ModelList:
            mname = df['Name']
            model.name
            if  model.name == f'protest_{mname}_{run_outcome}':
                print(f'Fitting {model.name}')

                print(datetime.now())
                model.fit_estimators(df['df'])
                print(datetime.now())

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

In [ ]:
# Predict.
if train:
    random.seed(1308)
    preds_dict = {}
    start_time = datetime.now()
    for df in datasets:
        for model in ModelList:
            mname = df['Name']
            if  model.name == f'protest_{mname}_{run_outcome}':
                print(f'Predicting for {mname}')
                print(datetime.now())
                preds_dict[model.name] = assign_into_df(df_from=model.predict(df['df']), df_to=df['df']).loc[periods_model[1].predict_start:periods_model[1].predict_end]
                print(datetime.now())

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

In [ ]:
# Evaluate.
if train:

    start_time = datetime.now()

    for df in datasets:
        for model in ModelList:
            mname = df['Name']
            if  model.name == f'protest_{mname}_{run_outcome}':
    
                print(f'Evaluating {model.name}')

                print(datetime.now())
                model.evaluate(df['df'])
                print(datetime.now())

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

In [ ]:
# Loading previously trained models based on the above defined outcome. 
if save_preds:
    for df in datasets:
        for model in ModelList:
            mname = df['Name']
            if  model.name == f'protest_{mname}_{run_outcome}':
                
                # Save models
                model.save(os.path.join(output_paths['models'],f'{mname}_{run_outcome}.joblib'))
                
    # Save dictonary.
    with open(os.path.join(output_paths['predictions'], f"preds_dict_{run_outcome}.p"), 'wb') as fp:
        pickle.dump(preds_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)
        
else:
    # Load models.
    ModelList = []
    models_to_load = models_to_train
    
    for m in models_to_load:
        ModelList.append(Model.load(os.path.join(output_paths['models'],f'{m}_{run_outcome}.joblib')))
    
    # Load dictonary.
    with open(os.path.join(output_paths['predictions'], f"preds_dict_{run_outcome}.p"), 'rb') as fp:
        preds_dict = pickle.load(fp)

## Evaluation <a class="anchor" id="evaluation"></a>

In [ ]:
importlib.reload(evaltools)

### Evaluation scores

In [ ]:
for model in ModelList:
    print(model.name)

In [ ]:
# Print scores for quick overview.
for model in ModelList:
    print(model.name)
    print(model.scores['B'][3]['uncalibrated']['average_precision'])
    print(model.scores['B'][6]['uncalibrated']['average_precision'])
    print(model.scores['B'][12]['uncalibrated']['average_precision'])
    print(model.scores['B'][36]['uncalibrated']['average_precision'])
    print(model.scores['B'][3]['uncalibrated']['area_under_roc'])
    print(model.scores['B'][6]['uncalibrated']['area_under_roc'])
    print(model.scores['B'][12]['uncalibrated']['area_under_roc'])
    print(model.scores['B'][36]['uncalibrated']['area_under_roc'])

In [ ]:
# Save all the scores to one df and one table. 
dfs_scores = []
for evalm in ['AP','AUROC','Brier']:
    df_scores = evaltools.df_eval_scores(
        preds_dict = preds_dict,
        model_list = models_to_train, 
        run_outcome = run_outcome,
        ev_name = evalm,
        depvar = depvar,
        steps = steps,
        round_to = 3, 
        path= os.path.join(output_paths['scores_tables'], f"eval_{evalm}_{run_outcome}.tex"
                          )
    )
    # Store in list.
    dfs_scores.append(df_scores)
    
# Rename
rename_models_dict = {
    f'baseline_simple':'M0',
    f'econ_nat_bl': 'M8 w/o pr',
    f'econ_full_bl': 'M9 w/o pr',
    f'inst_elecdemo_bl': 'M4 w/o pr',
    f'inst_civlib_bl' : 'M5 w/o pr',
    f'inst_elect_bl' : 'M6 w/o pr',
    f'inst_devi_bl': 'M7 w/o pr',
    f'inst_election_econ_national_bl{run_outcome}': 'M6M8 w/o pr',
    #f'polinst_election_econ_full_bl': 'M6M9 w/o pr',
    #f'polinst_devi_econ_national_bl': 'M7M8 w/o pr',
    #f'polinst_devi_econ_full_bl': 'M7M9 w/o pr',
    f'pr_naive_bl': 'M1',
    f'pr_dynamic_loc_bl': 'M2',
    f'pr_dynamic_nat_bl': 'M3',
    f'pr_elecdemo_bl': 'M4',
    f'pr_civlib_bl': 'M5',
    f'pr_elect_bl': 'M6',
    f'pr_devi_bl': 'M7',
    f'pr_econ_nat_bl': 'M8',
    f'pr_econ_full_bl': 'M9',
    #f'pr_polinst_election_econ_national_bl_{run_outcome}': 'M6M8',
    #f'pr_polinst_election_econ_full_bl{run_outcome}': 'M6M9',
    #f'pr_polinst_devi_econ_national_bl{run_outcome}': 'M7M8',
    #f'pr_polinst_devi_econ_full_bl{run_outcome}': 'M7M9'
}
reorder_models = [
    'M0',
    #'M0',
    'M1',
    'M2',
    'M3',
    'M4',
    'M4 w/o pr',
    'M5',
    'M5 w/o pr',
    'M6',
    'M6 w/o pr',
    'M7',
    'M7 w/o pr',
    'M8',
    'M8 w/o pr',
    'M9',
    'M9 w/o pr',
    'M6M8',
    'M6M8 w/o pr',
    'M6M9',
    'M6M9 w/o pr',
    'M7M8',
    'M7M8 w/o pr',
    'M7M9',
    'M7M9 w/o pr',
]

# Make into single df.
dfs_scores_all = pd.concat(dfs_scores,axis=1)
dfs_scores_all = dfs_scores_all.rename(index=rename_models_dict)
dfs_scores_all = dfs_scores_all.reindex(reorder_models).dropna()
dfs_scores_all.to_csv(os.path.join(output_paths['scores_tables'], f"eval_all_{run_outcome}.csv"))
print(dfs_scores_all)

# Write to tex. file. 
path = os.path.join(output_paths['scores_tables'], f"eval_all_{run_outcome}.tex")
tex = dfs_scores_all.reset_index().to_latex(index=False)

# Get meta infromation
now = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
meta = f"""
%Date: {now}
%Output created by protest_paper.ipynb.
%Compare eval metrics for all models.
\\
"""
tex = meta + tex
with open(path, "w") as f:
    f.write(tex)
print(f"Wrote scores table to {path}")

### Parallel coordinate plots

In [ ]:
# Make coordinate plots
dfs_coords = pd.read_csv(os.path.join(output_paths['scores_tables'], f"eval_all_{run_outcome}.csv"),index_col=[0],header=[0,1], skipinitialspace=True)
dfs_coords_copy = dfs_coords.swaplevel(axis=1)
dfs_coords_copy = dfs_coords_copy.reindex([('3', 'AP'),
 ('3', 'AUROC'),
 ('3', 'Brier'),
 ('6', 'AP'),
 ('6', 'AUROC'),
 ('6', 'Brier'),
 ('12', 'AP'),
 ('12', 'AUROC'),
 ('12', 'Brier'),
 ('36', 'AP'),
 ('36', 'AUROC'),
 ('36', 'Brier')], axis=1)
dfs_coords_copy = dfs_coords_copy[dfs_coords_copy.index.isin(['M0','M1','M2'])]

evaltools.plot_parcoord_allsteps(
    df = dfs_coords_copy,
    steps = ['3','6','12','36'],
    reverse = True,
    cmap='Dark2',
    legend_label=['M0','M1','M2',],
    path = os.path.join(output_paths["coord_plots"], f"coord_simple_{run_outcome}.png")
)

### Bootstrapping

In [ ]:
run_bs = False

In [ ]:
if run_bs:
    # Make list of all model names.
    all_model_names = []
    for model in ModelList:
        all_model_names.append(model.name)

    # Select models.
    coord_models_names = []
    matchers = [
        f'protest_baseline_simple_{run_outcome}',
        f'protest_pr_naive_bl_{run_outcome}',
        f'protest_pr_dynamic_loc_bl_{run_outcome}',
        f'protest_pr_dynamic_nat_bl_{run_outcome}',
        f'protest_inst_elecdemo_bl_{run_outcome}',
        f'protest_pr_elecdemo_bl_{run_outcome}',
        f'protest_inst_civlib_bl_{run_outcome}',
        f'protest_pr_civlib_bl_{run_outcome}',
        f'protest_inst_elect_bl_{run_outcome}',
        f'protest_pr_elect_bl_{run_outcome}',
        f'protest_inst_devi_bl_{run_outcome}',
        f'protest_pr_devi_bl_{run_outcome}',
        f'protest_econ_nat_bl_{run_outcome}',
        f'protest_pr_econ_nat_bl_{run_outcome}',
        f'protest_econ_full_bl_{run_outcome}',
        f'protest_pr_econ_full_bl_{run_outcome}'
    ]

    coord_models_names = [s for s in all_model_names if any(xs in s for xs in matchers)]

    models_to_boot = []
    for model in ModelList:
        if model.name in coord_models_names:
            models_to_boot.append(model.name)


    eval_fun = 'average_precision'
    steps = steps

    dfs_boots=[]
    for model_to_boot in models_to_boot:
        print(model_to_boot)
        for step in steps:
            print(step)
            df_boots = evaltools.boot_evalmetric(
                model_name = model_to_boot,
                preds_dict = preds_dict, 
                depvar = depvar,
                step=step,
                eval_fun = eval_fun,
                set_seed = 1308,
                n_bootstraps=1000,
            )

            dfs_boots.append(df_boots)

    df_boots_all = pd.concat(dfs_boots,axis=1) 
    df_boots_all.to_csv(os.path.join(output_paths["bootstrapped"], f"boots_ap_{run_outcome}.csv"))
    print('df written to', f"boots_ap_{run_outcome}.csv")
    
else:
    df_boots_all = pd.read_csv(os.path.join(output_paths["bootstrapped"], f"boots_ap_{run_outcome}.csv"))

In [ ]:
# Define parameters.

titles = [
    'M1 vs M0',
    'M2 vs M1',
    'M3 vs M1',
    'M2 vs M0',
    'M3 vs M0',
    'M4 vs M2',
    'M5 vs M2',
    'M6 vs M2',
    'M7 vs M2',
    'M8 vs M2',
    'M9 vs M2',
    'M4 vs M4 w/o pr ',
    'M5 vs M5 w/o pr',
    'M6 vs M6 w/o pr',
    'M7 vs M7 w/o pr',
    'M8 vs M8 w/o pr',
    'M9 vs M9 w/o pr',

]

model1 =[
    f'protest_pr_naive_bl_{run_outcome}_average_precision_', #M1
    f'protest_pr_dynamic_loc_bl_{run_outcome}_average_precision_', #M2
    f'protest_pr_dynamic_nat_bl_{run_outcome}_average_precision_', # M3
    f'protest_pr_dynamic_loc_bl_{run_outcome}_average_precision_', #M2
    f'protest_pr_dynamic_nat_bl_{run_outcome}_average_precision_', # M3
    f'protest_pr_elecdemo_bl_{run_outcome}_average_precision_', #M4
    f'protest_pr_civlib_bl_{run_outcome}_average_precision_', #M5
    f'protest_pr_elect_bl_{run_outcome}_average_precision_', #M6
    f'protest_pr_devi_bl_{run_outcome}_average_precision_', #M7
    f'protest_pr_econ_nat_bl_{run_outcome}_average_precision_', #M8
    f'protest_pr_econ_full_bl_{run_outcome}_average_precision_', #M9
    f'protest_pr_elecdemo_bl_{run_outcome}_average_precision_', #M4
    f'protest_pr_civlib_bl_{run_outcome}_average_precision_', #M5
    f'protest_pr_elect_bl_{run_outcome}_average_precision_', #M6
    f'protest_pr_devi_bl_{run_outcome}_average_precision_', #M7
    f'protest_pr_econ_nat_bl_{run_outcome}_average_precision_',
    f'protest_pr_econ_full_bl_{run_outcome}_average_precision_',
    

]

model2 = [
    f'protest_baseline_simple_{run_outcome}_average_precision_', # M0
    f'protest_pr_naive_bl_{run_outcome}_average_precision_', #M1
    f'protest_pr_naive_bl_{run_outcome}_average_precision_', #M1
    f'protest_baseline_simple_{run_outcome}_average_precision_', # M0
    f'protest_baseline_simple_{run_outcome}_average_precision_', # M0
    f'protest_pr_dynamic_loc_bl_{run_outcome}_average_precision_', # M2
    f'protest_pr_dynamic_loc_bl_{run_outcome}_average_precision_', # M2
    f'protest_pr_dynamic_loc_bl_{run_outcome}_average_precision_', # M2
    f'protest_pr_dynamic_loc_bl_{run_outcome}_average_precision_', # M2
    f'protest_pr_dynamic_loc_bl_{run_outcome}_average_precision_', # M2
    f'protest_pr_dynamic_loc_bl_{run_outcome}_average_precision_', # M2
    f'protest_inst_elecdemo_bl_{run_outcome}_average_precision_', #M4 wo pr
    f'protest_inst_civlib_bl_{run_outcome}_average_precision_',
    f'protest_inst_elect_bl_{run_outcome}_average_precision_',
    f'protest_inst_devi_bl_{run_outcome}_average_precision_',
    f'protest_econ_nat_bl_{run_outcome}_average_precision_',
    f'protest_econ_full_bl_{run_outcome}_average_precision_',  
]

In [ ]:
# Define parameters.
legendtrue = False

# Adjust these parameters for different outcomes 
if run_outcome == 'incidence':
    ymin = -0.02 
    ymax = 0.12

# Onset
if run_outcome == 'onset':
    ymin = -0.1 # Adjust these parameters for different outcomes 
    ymax = 0.14 # Adjust these parameters for different outcomes 

evaltools.plot_bootstrapped_diff(
    df=df_boots_all,
    titles=titles,
    modellist1=model1,
    modelllist2=model2,
    legendtrue=legendtrue,
    steps=steps,
    ymin=ymin,
    ymax=ymax,
    save_as=run_outcome,
    path_out=output_paths["bootstrapped"],
)

### Prediction maps

#### Probabilities

In [ ]:
# Fetch geometries from database
fetch_and_save_gdf = False
if fetch_and_save_gdf:
    gdf_mapping = organize.fetch_gdf()
    gdf_mapping = gdf_mapping.loc[445:480]
    gdf_mapping.to_csv(os.path.join(output_paths["data"], f"gdf_mapping.csv"))    
else:
    gdf_mapping = pd.read_csv(os.path.join(output_paths["data"], f"gdf_mapping.csv")).set_index(['month_id','priogrid_gid'])   

In [ ]:
import geopandas as gpd
from shapely import wkt
#import sqlalchemy as sa
#from ingester3.config import source_db_path

fetch_and_save_gdf = False
if fetch_and_save_gdf:

    engine = sa.create_engine(source_db_path)

    gdf_ci_master = gpd.GeoDataFrame.from_postgis(
        "SELECT id as country_id, in_africa, in_me, geom FROM prod.country",
        engine,
        geom_col='geom'
    )
    gdf_ci_master = gdf_ci_master.to_crs(4326)
    gdf_ci_master.to_csv(os.path.join(output_paths["data"], f"gdf_mapping_ci_master.csv")) 
else:
    gdf_ci_master = pd.read_csv(os.path.join(output_paths["data"], f"gdf_mapping_ci_master.csv"))
    gdf_ci_master['geom'] = gdf_ci_master['geom'].apply(wkt.loads)
    gdf_ci_master = gpd.GeoDataFrame(gdf_ci_master, crs="EPSG:4326", geometry='geom')
    


In [ ]:
# Match steps with months.
times = [447,450,456,480] 
allsteps = steps
times_steps = dict(zip(times, allsteps)) 

In [ ]:
# Define scale
proba_dict= {'0.1%':0.001, #'0.2%':0.002, '0.5%': 0.005,
               #'1%':0.01, 
             '2%':0.02, '5%': 0.05,
               '10%':0.1, '20%':0.2, '40%': 0.4,
               '60%':0.6, '80%':0.8, '90%': 0.9,
               '95%':0.95, '99%':0.99, 
              }

In [ ]:
for model in models_to_train:
    print(model)

In [ ]:
# Create full map.

for model in [models_to_train[0]]:
    
    mname = f'protest_{model}_{run_outcome}'
    print(mname)
    mname_plt = model.replace('simple_','')
    print(mname_plt)

    for key, value in times_steps.items():
        
        mapdf = pd.concat([preds_dict[mname][f'ss_{mname}_{value}'],preds_dict[mname][depvar],gdf_mappingt],axis=1)  
        mapdf['geometry'] = mapdf['geometry'].apply(wkt.loads)
        mapdf = gpd.GeoDataFrame(mapdf, geometry="geometry")
        print(key,value)
        
        pgm_masked=Mapper2(
            width=20,
            height=20,
            frame_on=True,
            bbox=bbox_from_cid_region('africa'),
        ).add_layer(
            gdf=mapdf.loc[key],
            map_dictionary = proba_dict,
            cmap = 'rainbow',
            transparency = 1,
            edgecolor="black",
            linewidth=0.5,
            column=f'ss_{mname}_{value}',
        ).add_layer(
            gdf=mapdf.loc[key][mapdf.loc[key][depvar]==1].geometry.centroid,
            marker="o",
            markersize=5,
            color="black"
        ).add_views_textbox(
            text=f'Run: protest \nModel: {model}, \nMonth: {key} (Step {value})',
            textsize=20)
        
        ax = pgm_masked.ax
        gdf_ci_master.plot(ax=ax,edgecolor='grey',linewidth=0.2,facecolor='None')

        pgm_masked.save(output_paths['maps']+f'/baseline_step{value}')

In [ ]:
# Zoom in on specific area.

for model in [models_to_train[0]]:
    
    mname = f'protest_{model}_{run_outcome}'
    print(mname)
    mname_plt = model.replace('simple_','')
    print(mname_plt)

    for key, value in times_steps.items():
        
        mapdf = pd.concat([preds_dict[mname][f'ss_{mname}_{value}'],preds_dict[mname][depvar],gdf_mapping],axis=1)  
        mapdf['geometry'] = mapdf['geometry'].apply(wkt.loads)
        mapdf = gpd.GeoDataFrame(mapdf, geometry="geometry")
        print(key,value)
        
        pgm_masked=Mapper2(
            width=20,
            height=20,
            frame_on=True,
            bbox=[22.6716, 48.819, -2.8909, 16.2484],
        ).add_layer(
            gdf=mapdf.loc[key],
            map_dictionary = proba_dict,
            cmap = 'rainbow',
            transparency = 1,
            edgecolor="black",
            linewidth=0.5,
            column=f'ss_{mname}_{value}',
        ).add_layer(
            gdf=mapdf.loc[key][mapdf.loc[key][depvar]==1].geometry.centroid,
            marker="o",
            markersize=5,
            color="black"
        ).add_views_textbox(
            text=f'Run: protest \nModel: {model}, \nMonth: {key} (Step {value})',
            textsize=20)
        
        ax = pgm_masked.ax
        gdf_ci_master.plot(ax=ax,edgecolor='grey',linewidth=0.2)

        pgm_masked.save(output_paths['maps']+f'/baseline_step{value}_zoom')

#### Map differences

In [ ]:
# Define the baseline model (to compute differnce)
baseline = True
baselinename = f'protest_baseline_simple_{run_outcome}'

for model in models_to_train:
    
    mname = f'protest_{model}_{run_outcome}'
    print(model)
    
    for s in steps:
        preds_dict[mname][f'diff_{mname}_{s}'] =  preds_dict[mname][f'ss_{mname}_{s}']-preds_dict[baselinename][f'ss_{baselinename}_{s}']

In [ ]:
# Define scale.
import matplotlib.colors as mcolors

proba_dict= {'-90%':-0.9, 
             '-60%':-0.6,
             '-40%':-0.4,
             '-20%':-0.2,
             '0%':0,
             '20%':0.2,
             '40%':0.4,
             '60%':0.6,
             '90%':0.9,
              }
cmap = plt.get_cmap('seismic')

In [ ]:
# Create full maps.
for model in models_to_train:
    
    mname = f'protest_{model}_{run_outcome}'
    print(mname)
    mname_plt = model.replace('simple_','')
    print(mname_plt)

    for key, value in times_steps.items():
        
        mapdf = pd.concat([preds_dict[mname][f'diff_{mname}_{s}'],preds_dict[mname][depvar],gdf_mapping],axis=1)  
        mapdf['geometry'] = mapdf['geometry'].apply(wkt.loads)
        mapdf = gpd.GeoDataFrame(mapdf, geometry="geometry")
        print(key,value)
        
        pgm_masked=Mapper2(
            width=20,
            height=20,
            frame_on=True,
            bbox=bbox_from_cid_region('africa'),
        ).add_layer(
            gdf=mapdf.loc[key],
            map_dictionary = proba_dict,
            cmap = cmap,
            transparency = 1,
            linewidth=0.5,
            column=f'diff_{mname}_{s}',
        ).add_layer(
            gdf=mapdf.loc[key][mapdf.loc[key][depvar]==1].geometry.centroid,
            marker="o",
            markersize=5,
            #column=f'actuals_step{value}',
            color="black"
        ).add_views_textbox(
            text=f'Run: protest \nModel: {model}, \nMonth: {key} (Step {value})',
            textsize=20)
        
        ax = pgm_masked.ax
        gdf_ci_master.plot(ax=ax,edgecolor='grey',linewidth=0.2,facecolor='None')

        pgm_masked.save(output_paths['maps']+f'/diff_{model}_step{value}')

In [ ]:
# Zoom in on specific area.

for model in models_to_train:
    
    mname = f'protest_{model}_{run_outcome}'
    print(mname)
    mname_plt = model.replace('simple_','')
    print(mname_plt)

    for key, value in times_steps.items():
        
        mapdf = pd.concat([preds_dict[mname][f'diff_{mname}_{s}'],preds_dict[mname][depvar],gdf_mapping],axis=1)  
        mapdf['geometry'] = mapdf['geometry'].apply(wkt.loads)
        mapdf = gpd.GeoDataFrame(mapdf, geometry="geometry")
        print(key,value)
        
        pgm_masked=Mapper2(
            width=20,
            height=20,
            frame_on=True,
            bbox=[22.6716, 48.819, -2.8909, 16.2484],
        ).add_layer(
            gdf=mapdf.loc[key],
            map_dictionary = proba_dict,
            cmap = cmap,
            transparency = 1,
            linewidth=0.5,
            column=f'diff_{mname}_{s}',
        ).add_layer(
            gdf=mapdf.loc[key][mapdf.loc[key][depvar]==1].geometry.centroid,
            marker="o",
            markersize=5,
            #column=f'actuals_step{value}',
            color="black"
        ).add_views_textbox(
            text=f'Run: protest \nModel: {model}, \nMonth: {key} (Step {value})',
            textsize=20)
        
        ax = pgm_masked.ax
        gdf_ci_master.plot(ax=ax,edgecolor='grey',linewidth=0.2,facecolor='None')

        pgm_masked.save(output_paths['maps']+f'/diff_{model}_step{value}_zoom')

### ICE/PDP plots

In [ ]:
for df in datasets:
    print(df['Name'])

In [ ]:
# Define parameters 
m = ModelList[14] # Confirm that this is the full pr_econ model.
pd_df = datasets[18]['df']
partition='B'
step = 3
featlist = [
    'decay_ts_6_acled_prex_dummy',
    'decay_ts_6_acled_prin_dummy',
    'decay_ts_6_acled_prpe_dummy',
    'decay_ts_6_acled_prri_dummy'
]

# Plot.
modelname = 'M9'
sample_n = 1000

# Change paramters here. For new estimation set "create_save_pickle = True"
save_fig = True
create_save_pickle = False

In [ ]:
# Create pickle
if create_save_pickle:
    for feat in featlist:
        print(feat)
        pd_output = partial_dependence(
            estimator = m.estimators.get(period_name=partition, step=step), 
            X = pd_df.loc[periods[1].train_start:periods[1].train_end][m.cols_features],
            features=feat, 
            response_method='auto', 
            percentiles=(0, 1), 
            grid_resolution=20,  
            kind='both',
        )


        pd_outputdict = dict(pd_output)
        a_file = open(output_paths['features'] + f"/pdp_{modelname}_s3_{feat}.pkl", "wb")
        pickle.dump(pd_outputdict, a_file)
        a_file.close()

In [ ]:
#Read pickle
featlist = [
    'decay_ts_6_acled_prex_dummy',
    'decay_ts_6_acled_prin_dummy',
    'decay_ts_6_acled_prpe_dummy',
    'decay_ts_6_acled_prri_dummy'
]

modelname = 'M9'
pd_df = datasets[18]['df']
sample_n = 1000
step = 3

save_fig = True

for feat in featlist:
    pd_output = pd.read_pickle(output_paths['features'] + f"/pdp_{modelname}_s3_{feat}.pkl")
    
    print('Making Plot.')
    fig, ax = plt.subplots(figsize=(10, 10))
    pdp_sample = pd.DataFrame(pd_output['individual'][0]).sample(n=sample_n,replace=True)
    pdp_sample = pdp_sample.apply(lambda row: row-pdp_sample.iloc[:, 0])
    plt.plot(pdp_sample.T,color='lightgrey',linewidth=0.5)
    ax.set_xlim(0, 1)

    xvals_cent = []
    for i in pd_output['average'][0]:
        xvals_cent.append(i-pd_output['average'][0][0])

    # Add average.
    plt.plot(pd_output['values'][0],xvals_cent,color='black')

    # Add horizontal line. 
    ax.axhline(y=0, color='dimgrey', linestyle='--', lw=2)

    # Add rug plot.
    y_min, y_max = (-0.15, 0.15)
    ax.plot(pd_df.loc[periods[1].train_start:periods[1].train_end][feat], [y_min]*len(pd_df.loc[periods[1].train_start:periods[1].train_end][feat]), '|', color='black',lw=0.5)

    ax.set_ylim(-0.15, 0.15)

    # Add title.
    plt.title(f'Centered ICE plot for {feat}\n {modelname}, step={step},\n sample size={sample_n}, grid points={len(pdp_sample.columns)}')
    plt.tight_layout()

    if save_fig:
        fig.savefig(output_paths['features'] + f"/{modelname}_s3_{feat}_adj.png",
                    dpi=200,
                    facecolor="white",
                    bbox_inches="tight",
                )

        plt.show()

### PR-Curves

In [ ]:
cm = plt.cm.get_cmap('Dark2')
colors = cm.colors
step = 3
fig_scale = 1


# Dictonary with model comparisons to be plotted.
dictoplots = [
    {
        'baseline_simple':'M0',
        'pr_naive_bl':'M1'
    },
    {
        'pr_naive_bl':'M1',
        'pr_dynamic_loc_bl':'M2'
    },
    {
        'pr_naive_bl':'M1',
        'pr_dynamic_nat_bl':'M3'
    },
    {
        'pr_dynamic_loc_bl':'M2',
        'pr_elecdemo_bl':'M4',
        'inst_elecdemo_bl':'M4 w/o pr'
    },
    {
        'pr_dynamic_loc_bl':'M2',
        'pr_civlib_bl':'M5',
        'inst_civlib_bl':'M5 w/o pr'
    },
    {
        'pr_dynamic_loc_bl':'M2',
        'pr_elect_bl':'M6',
        'inst_elect_bl':'M6 w/o pr'
    },
    {
        'pr_dynamic_loc_bl':'M2',
        'pr_devi_bl':'M7',
        'inst_devi_bl':'M7 w/o pr'
    },
    {
        'pr_dynamic_loc_bl':'M2',
        'pr_econ_nat_bl':'M8',
        'econ_nat_bl':'M8 w/o pr'
    },
    {
        'pr_dynamic_loc_bl':'M2',
        'pr_econ_full_bl':'M9',
        'econ_full_bl':'M9 w/o pr'
    },
]
    
for dic in dictoplots:

    namelist = []
    for i in dic.values():
        namelist.append(i)
    namelist = ''.join(namelist).replace('/','').replace(' ', '')

    # Figure
    fig = plt.figure(figsize=(8 * fig_scale, 8 * fig_scale))
    ax = fig.add_subplot(111)

    for model,clr, mname in zip(
        dic.keys(),colors, dic.values()):
        # Compute fpr, tpr, thresholds
        precision, recall, _ = precision_recall_curve(preds_dict[f'protest_{model}_{run_outcome}'][depvar], preds_dict[f'protest_{model}_{run_outcome}'][f'ss_protest_{model}_{run_outcome}_{step}'])

        #Plot
        plt.plot(recall,precision,label=f'{mname}',color=clr)
        plt.legend(title="Models")
        #plt.xlim([0.0, 1.0])
        #plt.ylim([0.0, 1.02])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        
        plt.savefig(
            output_paths['pr_curves'] + f"/pr_curve_{namelist}_s{step}.png", bbox_inches='tight',dpi=200)

### Bi-separation plots

In [ ]:
# Get gdf.
gdf_mapping = pd.read_csv(os.path.join(output_paths["data"], f"gdf_mapping.csv"))
gdf_mapping = gdf_mapping.rename(columns={'priogrid_gid':'pg_id'}).set_index(['month_id','pg_id']) 

gdf_filtered = df_filtered.set_index(['month_id','pg_id']).join(gdf_mapping,how='left')
gdf_filtered['geometry'] = gdf_filtered['geometry'].astype(str)
gdf_filtered['geometry'] = gdf_filtered['geometry'].apply(wkt.loads)
gdf_filtered = gpd.GeoDataFrame(gdf_filtered, geometry="geometry")
gdf_filtered['lon'] = gdf_filtered['geometry'].centroid.x
gdf_filtered['lat'] = gdf_filtered['geometry'].centroid.y

In [ ]:
time_s = [445,454,463,472]
time_e = [453,462,471,480]
steps_sp= 3
models1 = ['baseline_simple','pr_naive_bl','pr_naive_bl','pr_dynamic_loc_bl','pr_dynamic_loc_bl','pr_dynamic_loc_bl','pr_dynamic_loc_bl','pr_dynamic_loc_bl','pr_dynamic_loc_bl']
models2 = ['pr_naive_bl','pr_dynamic_loc_bl','pr_dynamic_nat_bl','pr_elecdemo_bl','pr_civlib_bl','pr_elect_bl','pr_devi_bl','pr_econ_nat_bl','pr_econ_full_bl']
models1_names = ['M0','M1','M1','M2','M2','M2','M2','M2','M2']
models2_names = ['M1','M2','M3','M4','M5','M6','M7','M8','M9']

df_filtered = preds_dict['protest_baseline_simple_incidence'].loc[450:452].reset_index().rename(columns={'priogrid_gid':'pg_id'})
filtered_bbox = gdf_filtered[(gdf_filtered.lat>-4.3) & (gdf_filtered.lat<16.4) & (gdf_filtered.lon>22) & (gdf_filtered.lon<50)]
filtered_bbox = filtered_bbox.reset_index()

for m1,m2,m1names,m2names in zip(models1,models2,models1_names,models2_names):
    for step in [steps_sp]:

        dfm1,dfm2 = preds_dict[f'protest_{m1}_{run_outcome}'].reset_index(),preds_dict[f'protest_{m2}_{run_outcome}'].reset_index()
        df_sp_1 = dfm1[dfm1['priogrid_gid'].isin(filtered_bbox.pg_id.unique())].set_index(['month_id','priogrid_gid']).drop(depvar,axis=1)
        df_sp_2 = dfm2[dfm2['priogrid_gid'].isin(filtered_bbox.pg_id.unique())].set_index(['month_id','priogrid_gid'])

        df_sp = pd.concat([df_sp_1,df_sp_2],axis=1)
        print('Plot')
        spl.BiseparationPlot(
            df = df_sp.loc[447],
            x = f'ss_protest_{m1}_{run_outcome}_{step}', 
            y = f'ss_protest_{m2}_{run_outcome}_{step}', 
            obs = f'{depvar}',
            lab = 'priogrid_gid',
            markersize=50,
            title = f"{m1names} versus {m2names}, step {step}",
            path = output_paths['bisep'] + f"/sp_{m1names}_{m2names}_s{step}_447.png"
        )